In [1]:
!pip install selenium

     |████████████████████████████████| 904 kB 5.4 MB/s eta 0:00:01


In [98]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

url = 'https://store.steampowered.com/search/?filter=topsellers'

#user_agent = {'User-agent': 'Mozilla/5.0'}
driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [47]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [48]:
table = soup.find('div', id='search_resultsRows')
table
rows = [row for row in table.find_all('a')]

In [49]:
len(rows)

50

In [50]:
#this was sufficient to scroll to the bottom of the top sellers page for steam, pulling code from scraping list
for i in range(500):
    #Scroll
    driver.execute_script(
        "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
    )
    time.sleep(1)

In [53]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [54]:
#successfully scraped 16.5k elements, hopefully all games.
table = soup.find('div', id='search_resultsRows')
table
rows = [row for row in table.find_all('a')]
rows[0]
len(rows)

16700

In [55]:
#pulling code created in 'scraping_list' notebook to create dataframe.
#dataframe index is game name, 

games = {}

for row in rows:
    title = (row.find('span', class_='title').text)
    url = row.get('href')
    release_date = (row.find('div', class_='col search_released responsive_secondrow').text)
    if (row.find('div', class_='col search_discount responsive_secondrow').text.strip()) == '':
        been_sale = 0
    else:
        been_sale = 1
    if been_sale == 0:
        orig_price = (row.find('div', class_='col search_price responsive_secondrow').text.strip())
    else:
        orig_price = (row.find('strike').text)
    games[title] = (url, 
                    release_date,
                    been_sale,
                    orig_price)
    

In [56]:
gamesdf1 = pd.DataFrame(games).T
gamesdf1.columns = ['link', 'release_date', 'been_sale', 'orig_price']

gamesdf1.shape

(16560, 4)

In [57]:
#no nulls!!
gamesdf1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16560 entries, OUTRIDERS to Dinosaur Hunt - Stegosaurus Expansion Pack
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   link          16560 non-null  object
 1   release_date  16560 non-null  object
 2   been_sale     16560 non-null  object
 3   orig_price    16560 non-null  object
dtypes: object(4)
memory usage: 646.9+ KB


In [58]:
gamesdf1.head()

,link,release_date,been_sale,orig_price
OUTRIDERS,https://store.steampowered.com/sub/504022/?snr...,"Apr 1, 2021",0,$59.99
Sea of Thieves,https://store.steampowered.com/app/1172620/Sea...,"Jun 3, 2020",1,$39.99
Halo: The Master Chief Collection,https://store.steampowered.com/sub/376282/?snr...,"Dec 3, 2019",1,$39.99
It Takes Two,https://store.steampowered.com/app/1426210/It_...,"Mar 26, 2021",0,$39.99
Valheim,https://store.steampowered.com/app/892970/Valh...,"Feb 2, 2021",0,$19.99


In [64]:
gamesdf1.tail()

,link,release_date,been_sale,orig_price
Dinosaur Hunt - Dragon Hunter Expansion Pack,https://store.steampowered.com/app/408102/Dino...,"Dec 18, 2015",0,$0.99
Dinosaur Hunt - Giant Spiders Hunter Expansion Pack,https://store.steampowered.com/app/431191/Dino...,"Mar 16, 2016",0,$0.99
Dinosaur Hunt - Brontosaurus Expansion Pack,https://store.steampowered.com/app/434942/Dino...,"Mar 16, 2016",0,$0.99
"Dinosaur Hunt - Vampires, Gargoyles, Mutants Hunter Expansion Pack",https://store.steampowered.com/app/431190/Dino...,"Jan 22, 2016",0,$0.99
Dinosaur Hunt - Stegosaurus Expansion Pack,https://store.steampowered.com/app/408100/Dino...,"Nov 13, 2015",0,$0.99


# attempt 1 to pull all game page data

Just "successfully" ran the function to pull all game data for the first 50 games. several nulls. going to run the code on here overnight, to see if there are enough non-null games out of the 16k to run a suitable model. if not, will have to write conditions for variance in page layouts, or supplement my scraped data with steamspy/steam's api

In [69]:
#attempting to pull my code in from  'scraping_page' to see if data can be pulled from from each game page.

def get_game_details(url):

    

    
    
    #Request HTML and parse
    response = driver.get(url)
    
    for i in range(5):
        #Scroll
        driver.execute_script(
            "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
        )
        time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, "html5lib")
    
    

    
    headers = ['title', 'num_reviews', 'review_rating', 
              'num_languages', 'tags', 'num_tags',
              'genres', 'num_genres', 'publisher']
    
    #Get title
    if not (soup.find('div', class_='apphub_AppName')):
        title = 'HELP'
    else:
        title = soup.find('div', class_='apphub_AppName').text

    #Get number of reviews
    if not (soup.find('div', id='Reviews_summary')):
        num_reviews = 'HELP'
    elif not (soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary')):
        num_reviews = 'HELP'
    else:
        rev_string = soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a').text
        num_reviews = rev_string.split()[2]
     

    #Get review rating
    if not (soup.find(class_='user_reviews_summary_bar')):
        rating = 'HELP'
    else:
        rating = soup.find(class_='user_reviews_summary_bar').find_all('span')[0].text
    
    
    #Get number of languages offered in
    if not soup.find('a', class_='all_languages'):
        num_languages = 'HELP'
    else:
        num_languages = soup.find('a', class_='all_languages').text.split()[2]
    

    #Get tags associated with game
    if not soup.find('div', class_='glance_tags'):
        tags_list = 'HELP'
    else:
        tags = soup.find('div', class_='glance_tags').find_all('a')
        tags_list = []
        for items in tags:
            tags_list.append(items.text.strip())
    
    
    #number of tags
    if not soup.find('div', class_='glance_tags'):
        num_tags = 'HELP'
    else:
        num_tags = len(tags_list)
    
    #genres list
    if not soup.find('div', class_='details_block'):
        genres_list = 'HELP'
    else:
        genres_all = soup.find('div', class_='details_block').find_all('a')[0:-2]
        genres_list = []
        for links in genres_all:
            genres_list.append(links.text)
        
    # number of genres
    if not soup.find('div', class_='details_block'):
        num_genres = 'HELP'
    else:
        num_genres = len(genres_list)
    
    #get publisher
    
    #if not soup.find('div', class_='details_block'): #this threw an error 10 pages in, going more general
    #    publisher = 'HALP PUBLISHER'
    #else:
    #    publisher = soup.find('div', class_='details_block').find_all('a')[-1].text
    
    
    if not soup.find('div', class_='glance_ctn_responsive_left'):
        publisher = 'HELP'
    else:
        publisher = soup.find('div', class_='glance_ctn_responsive_left').find_all('a')[-1].text
    
    #create dict of all games
    games_dict = dict(zip(headers, [title,
                                num_reviews,
                                rating,
                                num_languages, 
                                tags_list,
                                num_tags,
                                genres_list,
                                num_genres,
                                publisher]))

    return games_dict

In [70]:
games_info = []

for url in gamesdf1.link:
    games_info.append(get_game_details(url))
    

AttributeError: 'NoneType' object has no attribute 'text'

In [77]:
games_info = pd.DataFrame(games_info)

In [73]:
gamesdf1.iloc[312]

link            https://store.steampowered.com/app/10090/Call_...
release_date                                         Nov 18, 2008
been_sale                                                       0
orig_price                                                 $19.99
Name: Call of Duty: World at War, dtype: object

In [ ]:
# first attempt failed, but only after 300 iterations. going to look into nulls now, but am going
# to rewrite the function to account for failing at EVERY find, not just first one

In [80]:
import numpy as np
games_info.replace('HELP', np.nan, inplace=True)

In [81]:
games_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          257 non-null    object 
 1   num_reviews    207 non-null    object 
 2   review_rating  246 non-null    object 
 3   num_languages  197 non-null    object 
 4   tags           256 non-null    object 
 5   num_tags       256 non-null    float64
 6   genres         310 non-null    object 
 7   num_genres     310 non-null    float64
 8   publisher      256 non-null    object 
dtypes: float64(2), object(7)
memory usage: 22.0+ KB


# attempt #2
ran as I went to sleep, computer was asleep when I woke up, woke it up, program still running, assumed that it was running all night, but had to leave for bridal shower. terminated program, only ~400 titles copied. need to remove the sleep, takes too long between pages. although first time i tried, it it wasnt always scrolling to the bottom of the game page. will play with it tomorrow. 

In [83]:
#attempting to pull my code in from  'scraping_page' to see if data can be pulled from from each game page.

def get_game_details(url):

    

    
    
    #Request HTML and parse
    response = driver.get(url)
    
    for i in range(5):
        #Scroll
        driver.execute_script(
            "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
        )
        time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, "html5lib")
    
    

    
    headers = ['title', 'num_reviews', 'review_rating', 
              'num_languages', 'tags', 'num_tags',
              'genres', 'num_genres', 'publisher']
    
    #Get title
    if not (soup.find('div', class_='apphub_AppName')):
        title = 'HELP'
    else:
        title = soup.find('div', class_='apphub_AppName').text

    #Get number of reviews
    if not (soup.find('div', id='Reviews_summary')):
        num_reviews = 'HELP'
    elif not (soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary')):
        num_reviews = 'HELP'
    elif not soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a'):
        num_reviews = 'HELP'
    else:
        rev_string = soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a').text
        num_reviews = rev_string.split()[2]
     

    #Get review rating
    if not (soup.find(class_='user_reviews_summary_bar')):
        rating = 'HELP'
    elif not soup.find(class_='user_reviews_summary_bar').find_all('span'):
        rating = 'HELP'
    else:
        rating = soup.find(class_='user_reviews_summary_bar').find_all('span')[0].text
    
    
    #Get number of languages offered in
    if not soup.find('a', class_='all_languages'):
        num_languages = 'HELP'
    else:
        num_languages = soup.find('a', class_='all_languages').text.split()[2]
    

    #Get tags associated with game
    if not soup.find('div', class_='glance_tags'):
        tags_list = 'HELP'
    else:
        tags = soup.find('div', class_='glance_tags').find_all('a')
        tags_list = []
        for items in tags:
            tags_list.append(items.text.strip())
    
    
    #number of tags
    if not soup.find('div', class_='glance_tags'):
        num_tags = 'HELP'
    else:
        num_tags = len(tags_list)
    
    #genres list
    if not soup.find('div', class_='details_block'):
        genres_list = 'HELP'
    else:
        genres_all = soup.find('div', class_='details_block').find_all('a')[0:-2]
        genres_list = []
        for links in genres_all:
            genres_list.append(links.text)
        
    # number of genres
    if not soup.find('div', class_='details_block'):
        num_genres = 'HELP'
    else:
        num_genres = len(genres_list)
    
    #get publisher
    
    #if not soup.find('div', class_='details_block'): #this threw an error 10 pages in, going more general
    #    publisher = 'HALP PUBLISHER'
    #else:
    #    publisher = soup.find('div', class_='details_block').find_all('a')[-1].text
    
    
    if not soup.find('div', class_='glance_ctn_responsive_left'):
        publisher = 'HELP'
    else:
        publisher = soup.find('div', class_='glance_ctn_responsive_left').find_all('a')[-1].text
    
    #create dict of all games
    games_dict = dict(zip(headers, [title,
                                num_reviews,
                                rating,
                                num_languages, 
                                tags_list,
                                num_tags,
                                genres_list,
                                num_genres,
                                publisher]))

    return games_dict

In [84]:
games_info = []

for url in gamesdf1.link:
    games_info.append(get_game_details(url))

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=89.0.4389.114)


In [85]:
len(games_info)

406

In [87]:
games_info = pd.DataFrame(games_info)

In [93]:
games_info.replace(np.nan, null)

NameError: name 'null' is not defined

In [90]:
games_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          406 non-null    object
 1   num_reviews    406 non-null    object
 2   review_rating  406 non-null    object
 3   num_languages  406 non-null    object
 4   tags           406 non-null    object
 5   num_tags       406 non-null    object
 6   genres         406 non-null    object
 7   num_genres     406 non-null    object
 8   publisher      406 non-null    object
dtypes: object(9)
memory usage: 28.7+ KB


In [94]:
games_info.head()

,title,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
0,HELP,HELP,HELP,HELP,HELP,HELP,"[Action, Adventure, RPG, People Can Fly]",4,HELP
1,Sea of Thieves,"109,916",Very Positive,13,"[Adventure, Multiplayer, Open World, Pirates, ...",20,"[Action, Adventure]",2,Xbox Game Studios
2,HELP,HELP,HELP,HELP,HELP,HELP,"[Action, 343 Industries, Splash Damage, Ruffia...",8,HELP
3,It Takes Two,"10,433",Overwhelmingly Positive,12,"[Co-op, Adventure, 3D Platformer, Puzzle, Puzz...",20,"[Action, Adventure, Hazelight]",3,Electronic Arts
4,Valheim,"215,071",Overwhelmingly Positive,12,"[Open World Survival Craft, Online Co-Op, Open...",20,"[Action, Adventure, Indie, RPG, Early Access, ...",6,Coffee Stain Publishing


# attempt number 3 

going to add to the scoll, but take awawy the sleep, hoping that it will cycle through the site faster

In [99]:
#attempting to pull my code in from  'scraping_page' to see if data can be pulled from from each game page.

def get_game_details(url):

    

    
    
    #Request HTML and parse
    response = driver.get(url)
    
    for i in range(7):
        #Scroll
        driver.execute_script(
            "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
        )
        #time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, "html5lib")
    
    

    
    headers = ['title', 'num_reviews', 'review_rating', 
              'num_languages', 'tags', 'num_tags',
              'genres', 'num_genres', 'publisher']
    
    #Get title
    if not (soup.find('div', class_='apphub_AppName')):
        title = 'HELP'
    else:
        title = soup.find('div', class_='apphub_AppName').text

    #Get number of reviews
    if not (soup.find('div', id='Reviews_summary')):
        num_reviews = 'HELP'
    elif not (soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary')):
        num_reviews = 'HELP'
    elif not soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a'):
        num_reviews = 'HELP'
    else:
        rev_string = soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a').text
        num_reviews = rev_string.split()[2]
     

    #Get review rating
    if not (soup.find(class_='user_reviews_summary_bar')):
        rating = 'HELP'
    elif not soup.find(class_='user_reviews_summary_bar').find_all('span'):
        rating = 'HELP'
    else:
        rating = soup.find(class_='user_reviews_summary_bar').find_all('span')[0].text
    
    
    #Get number of languages offered in
    if not soup.find('a', class_='all_languages'):
        num_languages = 'HELP'
    else:
        num_languages = soup.find('a', class_='all_languages').text.split()[2]
    

    #Get tags associated with game
    if not soup.find('div', class_='glance_tags'):
        tags_list = 'HELP'
    else:
        tags = soup.find('div', class_='glance_tags').find_all('a')
        tags_list = []
        for items in tags:
            tags_list.append(items.text.strip())
    
    
    #number of tags
    if not soup.find('div', class_='glance_tags'):
        num_tags = 'HELP'
    else:
        num_tags = len(tags_list)
    
    #genres list
    if not soup.find('div', class_='details_block'):
        genres_list = 'HELP'
    else:
        genres_all = soup.find('div', class_='details_block').find_all('a')[0:-2]
        genres_list = []
        for links in genres_all:
            genres_list.append(links.text)
        
    # number of genres
    if not soup.find('div', class_='details_block'):
        num_genres = 'HELP'
    else:
        num_genres = len(genres_list)
    
    #get publisher
    
    #if not soup.find('div', class_='details_block'): #this threw an error 10 pages in, going more general
    #    publisher = 'HALP PUBLISHER'
    #else:
    #    publisher = soup.find('div', class_='details_block').find_all('a')[-1].text
    
    
    if not soup.find('div', class_='glance_ctn_responsive_left'):
        publisher = 'HELP'
    else:
        publisher = soup.find('div', class_='glance_ctn_responsive_left').find_all('a')[-1].text
    
    #create dict of all games
    games_dict = dict(zip(headers, [title,
                                num_reviews,
                                rating,
                                num_languages, 
                                tags_list,
                                num_tags,
                                genres_list,
                                num_genres,
                                publisher]))

    return games_dict

In [100]:
games_info = []

for url in gamesdf1.link:
    games_info.append(get_game_details(url))

KeyboardInterrupt: 

In [102]:
len(games_info)

46

In [ ]:
'''
SO, stopped after 3/5 minutes. it would go through 3/4 pages lightning fast, then would pause for a while to
load. could be deanas internet, have never worked here before. watching the pages, it did not appear that the
pages had enough time (when not loading) to get to the bottom which is where the total numbr of reviews comes in.
going to see number of num reviews pulled from this test.
MOVING FORWARD i will save 2 dfs at a time, post and current, so I can compare to last iteration. should
have thought of that already.
'''

In [104]:
games_info = pd.DataFrame(games_info)

In [105]:
games_info

,title,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
0,HELP,HELP,HELP,HELP,HELP,HELP,"[Action, Adventure, RPG, People Can Fly]",4,HELP
1,Sea of Thieves,HELP,Very Positive,13,"[Adventure, Multiplayer, Open World, Pirates, ...",20,"[Action, Adventure]",2,Xbox Game Studios
2,HELP,HELP,HELP,HELP,HELP,HELP,"[Action, 343 Industries, Splash Damage, Ruffia...",8,HELP
3,It Takes Two,HELP,Overwhelmingly Positive,12,"[Co-op, Adventure, 3D Platformer, Puzzle, Puzz...",20,"[Action, Adventure, Hazelight]",3,Electronic Arts
4,Valheim,HELP,Overwhelmingly Positive,12,"[Open World Survival Craft, Online Co-Op, Open...",20,"[Action, Adventure, Indie, RPG, Early Access, ...",6,Coffee Stain Publishing
5,Horizon Zero Dawn™ Complete Edition,HELP,Mostly Positive,20,"[Adventure, Open World, Action, Female Protago...",20,"[Action, Adventure, RPG]",3,"PlayStation Mobile, Inc."
6,HELP,HELP,HELP,HELP,HELP,HELP,"[Action, Adventure, Indie, RPG, Nicalis, Inc.,...",6,HELP
7,HELP,HELP,HELP,HELP,HELP,HELP,"[RPG, Nihon Falcom, Engine Software BV, PH3 Gm...",5,HELP
8,HELP,HELP,HELP,HELP,HELP,HELP,"[Action, FromSoftware, Inc., FromSoftware, Inc.]",3,HELP
9,HELP,HELP,HELP,HELP,HELP,HELP,"[Action, Adventure, Massively Multiplayer, Rar...",4,HELP


In [106]:
prev_df = games_info.copy()

# attempt 4
inverse attempt, fewer scrolls, but leaving the sleep, 